Get the Cookies and headers More like a people.

In [3]:
from seleniumwire import webdriver  
from fake_useragent import UserAgent
from selenium.webdriver.chrome.options import Options


# - step1 selenium 上網登入 ee-class
url = "https://military.ncu.edu.tw" # = ee-class url

# : user agent setting
ua = UserAgent()
user_agent = ua.random

# : option setting
chrome_opt = Options()
chrome_opt.add_argument("start-maximized")  # 開啟視窗最大化
chrome_opt.add_argument("--disable-proxy-certificate-handler")
chrome_opt.add_argument("--disable-content-security-policy")
chrome_opt.add_argument("--disable-proxy-certificate-handler")
chrome_opt.add_argument("--ignore-certificate-errors")
chrome_opt.add_argument("--incognito")      # 設置隱身模式，可以避免個人化廣告，加速網頁瀏覽

# : driver setting 
driver = webdriver.Chrome(options = chrome_opt) # => insert the option to the driver (can be overridden)
driver.delete_all_cookies() # 防止污染

# : Get the Cookies & Headers
current_login_cookie = driver.get_cookies()
current_login_requests = driver.requests

headers = {}
for request in driver.requests:
    if f"https://ncueeclass.ncu.edu.tw/dashboard" == str(request):
        headers = {k: v for k, v in request.headers.items()} 
        
# 自動填入 headers
print("cookies:", current_login_cookie)
print("requests:", current_login_requests)


Request the website Content

In [1]:
import requests
from bs4 import BeautifulSoup

#: default setting.
url = "https://military.ncu.edu.tw"

try:
    with requests.Session() as session: 
    # ~ Session can update the performance of the server. 
    
        resp = session.get(url)
        # ~ Get the response of the website.
        
        # @ Check the response and Parser the response to HTML.
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        
        # @ Get the elements of the site.
        title = soup.title.get_text() if soup.title else ''
        print("網頁標題:", title)
        
except requests.HTTPError as e:
    print('HTTP error', e.response.status_code)
except requests.RequestException as e:
    print('Request error:', e)

網頁標題: 歡迎光臨國立中央大學校園安全中心
